In [66]:
import pathops

from fontTools import ttLib

def extract_glyph_set(font_path):
    font = ttLib.TTFont(font_path)
    return font.getGlyphSet()

def skPathFromGlyph(glyphName: str, glyphSet) -> pathops.Path:
    path = pathops.Path()
    pathPen = path.getPen(glyphSet=glyphSet)
    glyphSet[glyphName].draw(pathPen)
    return path

In [161]:
font_names = [
    'glametrix.otf',
    'mikodacs.otf',
    'rawengulk.otf',
    'hussarbold.otf',
    'resamitz.otf',
]

glyphs = {name[:-4]: extract_glyph_set(f'./fonts/{name}') for name in font_names}


In [180]:
path = skPathFromGlyph('J', glyphs['resamitz'])

In [205]:
def normalize_path(path, invert_y=False):
    llx, lly, urx, ury = path.bounds
    height = ury - lly
    mid_x = 0.5 * (llx + urx) / height
    mid_y = 0.5 * (lly + ury) / height
    scale = 1.0 / height
    return transform_segments(path.segments, scale=scale, x_offset=-mid_x, y_offset=-mid_y, invert_y=invert_y)


def transform_segments(segments, scale=1.0, x_offset=0.0, y_offset=0.0, invert_y=False):
    def _x(v):
        return scale * v + x_offset
    def _y(v):
        if invert_y:
            return -1.0 * (scale * v + y_offset)
        else:
            return scale * v + y_offset
    return [(c, [(_x(p[0]), _y(p[1])) for p in d]) for c, d in path.segments]

def roll_segments(segments, num_places):
    roll = num_places if num_places >= 0 else num_places - 1
    if segments[roll][0] not in ['lineTo', 'moveTo']:
        raise ValueError(f'Unable to roll {roll}, because starting element is {segments[roll][0]}')
    if roll:
        segments[0] = ('lineTo', segments[0][1])
        segments = segments[roll:-1] + segments[:roll] + segments[-1:]
        segments[0] = ('moveTo', segments[0][1])
    return segments

def print_segments(segments):
    def pt(v):
        return f'{v[0]}, {v[1]}'
    print('beginShape();')
    move_to = (0, 0)
    for cmd, point in segments:
        if cmd == 'curveTo':
            print(f'bezierVertex({pt(point[0])}, {pt(point[1])}, {pt(point[2])});')
        elif cmd == 'lineTo':
            print(f'vertex({pt(point[0])});')
        elif cmd == 'moveTo':
            move_to = point[0]
            print(f'vertex({pt(point[0])});')
        elif cmd == 'closePath':
            print(f'vertex({pt(move_to)});')
            print('endShape();')
        else:
            print(f'===={cmd}{point}')

def debug_segments(segments):
    def pt(v):
        return f'{v[0]:6.3f}, {v[1]:6.3f}'
    print('beginShape();')
    move_to = (0, 0)
    for i, (cmd, point) in enumerate(segments):
        if cmd == 'curveTo':
            print(f'{i:3d} b {pt(point[0])} {pt(point[1])} {pt(point[2])}')
        elif cmd == 'lineTo':
            print(f'{i:3d} v {pt(point[0])}')
        elif cmd == 'moveTo':
            move_to = point[0]
            print(f'{i:3d} m {pt(point[0])}')
        elif cmd == 'closePath':
            print(f'{i:3d} c {pt(move_to)}')
            print('endShape();')
        else:
            print(f'===={cmd}{point}')

def pseudocode_segments(segments):
    print('beginShape();')
    points = []
    for cmd, point in segments:
        index = len(points)
        if cmd == 'curveTo':
            print(f'interpolateBezierVertex({index}, {index + 1}, {index + 2}, v0, v1, t);')
            points.extend([list(p) for p in point])
        elif cmd in ['lineTo', 'moveTo']:
            print(f'interpolateVertex({index}, v0, v1, t);')
            points.extend([list(p) for p in point])
        elif cmd == 'closePath':
            print('interpolateVertex(0, v0, v1, t);')
            print('endShape();')
        else:
            print(f'===={cmd}{point}')
    return points

In [ ]:
roll_places = {
    'glametrix':0,
    'mikodacs':-1,
    'rawengulk':-6,
    'hussarbold':0,
    'resamitz':-1,
}

for key in glyphs:
    print(f'=== {key} ===')
    path = skPathFromGlyph('J', glyphs[key])
    # print_segments(roll_segments(normalize_path(path, True), roll_places[key]))
    debug_segments(roll_segments(normalize_path(path, True), roll_places[key]))
    # print(pseudocode_segments(roll_segments(normalize_path(path, True), roll_places[key])))

In [ ]:
roll_places = {
    'glametrix':0,
    'mikodacs':0,
    'rawengulk':0,
    'hussarbold':-1,
    'resamitz':-2,
}

for key in glyphs:
    print(f'=== {key} ===')
    path = skPathFromGlyph('T', glyphs[key])
    # print_segments(roll_segments(normalize_path(path, True), roll_places[key]))
    # debug_segments(roll_segments(normalize_path(path, True), roll_places[key]))
    print(pseudocode_segments(roll_segments(normalize_path(path, True), roll_places[key])))

In [ ]:
def block_shifts(segments, shifts):
    for shift in shifts:
        segments = block_shift(segments, *shift)
    return segments

def block_shift(segments, i_from, i_to, size):
    i_end = i_from + size
    if i_from > i_to:
        return segments[:i_to] + segments[i_from:i_end] + segments[i_to:i_from] + segments[i_end:]
    elif i_from < i_to:
        return segments[:i_from] + segments[i_end:i_to] + segments[i_from:i_end] + segments[i_to:i_from]
    else:
        return segments

def block_rolls(segments, rolls):
    for roll in rolls:
        segments = block_roll(segments, *roll)
    return segments

def block_roll(segments, i_start, i_end, num_places):
    return segments[:i_start] + roll_segments(segments[i_start:i_end], num_places) + segments[i_end:]

def insert_node_after(segments, index):
    end = index + 1
    return segments[:end] + [('lineTo', [segments[index][1][-1]])] + segments[end:]

def insert_nodes_after(segments, indices):
    indices.sort()
    indices.reverse()
    for index in indices:
        segments = insert_node_after(segments, index)
    return segments



shift_blocks = {
    'glametrix':[],
    'mikodacs':[(0,24,7),],
    'rawengulk':[(12,0,10),(16,10,6),],
    'hussarbold':[(14,8,6),],
    'resamitz':[(14,8,6),],
}

roll_blocks = {
    'glametrix':[(0,8,-1),(14,20,1),],
    'mikodacs':[(10,17,1),(17,24,1)],
    'rawengulk':[(16,22,1)],
    'hussarbold':[(8,14,1),(14,20,1)],
    'resamitz':[],
}

inserted_indices = {
    'glametrix':[3,4,4,4,5,10,16],
    'mikodacs':[5,5,5],
    'rawengulk':[18,12,7,2,1],
    'hussarbold':[16,10,3,4,4,4,5],
    'resamitz':[16,10,3,4,4,4,5],
}

for key in glyphs:
    path = skPathFromGlyph('B', glyphs[key])
    parsed_segments = insert_nodes_after(
        block_rolls(
            block_shifts(normalize_path(path, True), 
                        shift_blocks[key]),
            roll_blocks[key]),
        inserted_indices[key])
    print(f'=== {key} ===')
    debug_segments(parsed_segments)
    # print(pseudocode_segments(parsed_segments))

In [206]:
shift_blocks = {
    'glametrix':[],
    'mikodacs':[(0,24,7),],
    'rawengulk':[(12,0,10),(16,10,6),],
    'hussarbold':[(14,8,6),],
    'resamitz':[(14,8,6),],
}

roll_blocks = {
    'glametrix':[(0,8,-1),(14,20,1),],
    'mikodacs':[(10,17,1),(17,24,1)],
    'rawengulk':[(0,9,4),(16,22,1)],
    'hussarbold':[(8,14,1),(14,20,1)],
    'resamitz':[],
}

inserted_indices = {
    'glametrix':[16,10,5,3,3],
    'mikodacs':[3],
    'rawengulk':[18,12,7],
    'hussarbold':[16,10,5,3,3],
    'resamitz':[16,10,5,3,3],
}

for key in glyphs:
    path = skPathFromGlyph('B', glyphs[key])
    # parsed_segments = block_rolls(
    #         block_shifts(normalize_path(path, True), 
    #                     shift_blocks[key]),
    #         roll_blocks[key])
    parsed_segments = insert_nodes_after(
        block_rolls(
            block_shifts(normalize_path(path, True), 
                        shift_blocks[key]),
            roll_blocks[key]),
        inserted_indices[key])
    print(f'=== {key} ===')
    debug_segments(parsed_segments)
    # print(pseudocode_segments(parsed_segments))

=== glametrix ===
beginShape();
  0 m -0.339, -0.500
  1 v -0.339,  0.500
  2 v -0.026,  0.500
  3 b  0.193,  0.500  0.339,  0.392  0.339,  0.229
  4 v  0.339,  0.229
  5 v  0.339,  0.229
  6 b  0.339,  0.106  0.218,  0.002  0.056, -0.025
  7 b  0.184, -0.058  0.276, -0.146  0.276, -0.250
  8 v  0.276, -0.250
  9 b  0.276, -0.400  0.139, -0.500 -0.068, -0.500
 10 c -0.339, -0.500
endShape();
 11 m -0.203,  0.031
 12 v -0.036,  0.031
 13 b  0.090,  0.031  0.193,  0.120  0.193,  0.229
 14 v  0.193,  0.229
 15 b  0.193,  0.329  0.104,  0.396 -0.028,  0.396
 16 v -0.203,  0.396
 17 c -0.203,  0.031
endShape();
 18 m -0.203, -0.396
 19 v -0.068, -0.396
 20 b  0.064, -0.396  0.151, -0.338  0.151, -0.250
 21 v  0.151, -0.250
 22 b  0.151, -0.152  0.053, -0.073 -0.068, -0.073
 23 v -0.203, -0.073
 24 c -0.203, -0.396
endShape();
=== mikodacs ===
beginShape();
  0 m -0.333, -0.500
  1 v -0.333,  0.500
  2 v  0.014,  0.500
  3 b  0.247,  0.500  0.333,  0.456  0.333,  0.333
  4 v  0.333,  0.333
 

In [207]:
w = 392 / 200.0
h = 212 / 200.0
print(f'w={w} h={h}')

w=1.96 h=1.06
